In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

In [36]:
from pyspark.sql.types import *

### spark 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

### load_tables 

In [6]:
%%time
clickstream = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_CLICKSTREAM, hive).cache()

CPU times: user 674 µs, sys: 2.18 ms, total: 2.86 ms
Wall time: 62 ms


In [4]:
%%time
ratings = load_table(SBX_TEAM_DIGITCAMP, ALS_ASUP_RATINGS_SCALLED, hive).cache()

CPU times: user 2.1 ms, sys: 107 µs, total: 2.2 ms
Wall time: 69.3 ms


In [7]:
clickstream.count()

167939818

In [27]:
ratings.show()

,inn,prod_cd_asup,ratings,load_dt
0,0012345678,korpkarta,0.7311,2021-12-14
1,0012345678,rabota_ru,0.6733,2021-12-14
2,0012345678,kredit,0.6687,2021-12-14
3,0012345678,e_inv,0.6544,2021-12-14
4,0012345678,merchant-acquiring,0.6333,2021-12-14
5,0012345678,credit_card,0.6252,2021-12-14
6,0012345678,bip,0.6222,2021-12-14
7,0012345678,zarplata,0.6067,2021-12-14
8,0012345678,LYurist,0.5858,2021-12-14
9,0012345678,sppu,0.5849,2021-12-14


###### story_click 

In [20]:
sbbol_story_click = clickstream.filter("eventCategory like '%[operations]: offers%'")\
                               .filter("eventAction like 'open card'")\
                               .filter("eventLabel like '%[placement: main[main_feed]]%'")\
                               .filter("product <> 'none'").cache()

In [21]:
story_click_cnt = sbbol_story_click.groupBy("product").agg(f.count("sbbolUserId").alias("story_click_cnt"),
                                                           f.countDistinct("sbbolUserId").alias("story_click_dist_cnt"))\
                                   .orderBy(f.col("story_click_cnt").desc()).cache()

In [40]:
story_click_cnt_all = int(story_click_cnt.toPandas()["story_click_dist_cnt"].sum())

In [41]:
story_click_cnt_all

30687

In [24]:
story_click_cnt.show(54)

,product,story_click_cnt,story_click_dist_cnt
0,korpkarta,8717,7185
1,kredit,6590,5185
2,credit_card,4771,3995
3,merchant-acquiring,3573,3094
4,smart_credit,2018,1759
5,zarplata,1646,1400
6,e-acquiring,1449,1251
7,e_inv,1312,1174
8,lyurist,908,808
9,rabota_ru,786,670


In [28]:
now_time = datetime.fromtimestamp(time.time())

In [29]:
now_time

datetime.datetime(2021, 12, 14, 10, 53, 41, 135248)

In [39]:
int(story_click_cnt_all)

30687

In [43]:
story_click_popular = story_click_cnt.filter("story_click_dist_cnt > 300")\
                                     .select(f.col("product").alias("prod_cd_asup"),
                                             (f.col("story_click_dist_cnt") / f.lit(story_click_cnt_all)).alias("ratings"),
                                             f.lit(f.current_date().cast(StringType())).alias("load_dt"))

In [44]:
story_click_popular.show()

,prod_cd_asup,ratings,load_dt
0,korpkarta,0.234138,2021-12-14
1,kredit,0.168964,2021-12-14
2,credit_card,0.130185,2021-12-14
3,merchant-acquiring,0.100824,2021-12-14
4,smart_credit,0.057321,2021-12-14
5,zarplata,0.045622,2021-12-14
6,e-acquiring,0.040766,2021-12-14
7,e_inv,0.038257,2021-12-14
8,lyurist,0.026330,2021-12-14
9,rabota_ru,0.021833,2021-12-14


###### story_show 

не работает в текущий момент, необходимо внести изменения в пайплайн

In [9]:
sbbol_story_show = clickstream.filter("eventCategory like '%[operations]: offers%'")\
                                      .filter("eventAction like '%impression-story%'")\
                                      .filter("eventLabel like '%[placement: main[main_feed]]%'")